In [40]:
import json
import os
import re

import matplotlib.pyplot as plt
import seaborn as sns

In [41]:
ALL_MODELS_FOLDER = "../models/"

In [42]:
bpc_losses = {}
eval_losses = {}

# Traverse through all folders and read trainer_state.json files
for subdir, _, files in os.walk(ALL_MODELS_FOLDER):
    if "trainer_state.json" in files:
        file_path = os.path.join(subdir, "trainer_state.json")
        with open(file_path, "r") as f:
            data = json.load(f)
            # Collect all eval_bpc values in a list
            bpc_list = []
            eval_loss_list = []
            for log_entry in data.get("log_history", []):
                if "eval_bpc" in log_entry:
                    bpc_list.append(log_entry["eval_bpc"])
                elif if "eval_loss" in log_entry:
                    eval_loss_list.append(log_entry["eval_loss"])

            # Extract only the model name from the subdirectory path
            match = re.search(r"models[\\/](.*?)(\\|/|$)", subdir)
            if match:
                model_name = match.group(1)
                if bpc_list:
                    bpc_losses[model_name] = bpc_list

In [44]:
bpc_losses

{'model_es_BPE_vs10000': [3.9953058180117345,
  3.815965256450828,
  3.6624676062772807,
  3.481875157404405,
  3.315451010890085,
  3.0786520799016173,
  3.0615162639338633,
  2.991322781436518,
  2.9219343985406874,
  2.9342840626559132,
  2.920990074048429,
  2.8923648238112234,
  2.9088227385815504,
  2.8587452818780017,
  2.831865208228805,
  2.8834524070248495,
  2.7975733162665746,
  2.8040014855765003,
  2.8111562262478,
  2.7733700315133722,
  2.757835074782902,
  2.768960140978562,
  2.73191926615214,
  2.769781908432716,
  2.728348755105268,
  2.7338062432001213,
  2.734358180148394,
  2.7231005502507446,
  2.7593194678475075,
  2.695736842111602,
  2.7494148338223803,
  2.709901806599649,
  2.7272720019433665,
  2.7156565129203374,
  2.7293385863560506,
  2.763826265201652,
  2.68039728174644,
  2.7158696393102053,
  2.753272422638411,
  2.68973129989913,
  2.695348935977371,
  2.733073509204307,
  2.74277574325526,
  2.6612346010851176,
  2.6915696649563388,
  2.7080894752

In [27]:
with open(ALL_MODELS_FOLDER + 'model_es_BPE_vs10000/checkpoint-583/trainer_state.json', "r") as f:
    data = json.load(f)
    

In [30]:
data.get('log_history')

[{'epoch': 0.008576329331046312,
  'grad_norm': 14.54792308807373,
  'learning_rate': 8.333333333333333e-05,
  'loss': 9.2037,
  'step': 5},
 {'epoch': 0.008576329331046312,
  'eval_bpc': 3.9953058180117345,
  'eval_loss': 9.072009086608887,
  'eval_perplexity': 8705.6943359375,
  'eval_runtime': 10.423,
  'eval_samples_per_second': 245.61,
  'eval_steps_per_second': 3.838,
  'step': 5},
 {'epoch': 0.017152658662092625,
  'grad_norm': 12.351099014282227,
  'learning_rate': 0.00016666666666666666,
  'loss': 8.9551,
  'step': 10},
 {'epoch': 0.017152658662092625,
  'eval_bpc': 3.815965256450828,
  'eval_loss': 8.755976676940918,
  'eval_perplexity': 6166.81787109375,
  'eval_runtime': 10.3346,
  'eval_samples_per_second': 247.712,
  'eval_steps_per_second': 3.871,
  'step': 10},
 {'epoch': 0.025728987993138937,
  'grad_norm': 12.194334983825684,
  'learning_rate': 0.00025,
  'loss': 8.633,
  'step': 15},
 {'epoch': 0.025728987993138937,
  'eval_bpc': 3.6624676062772807,
  'eval_loss': 8.

In [25]:
[log['eval_bpc'] for log in data['log_history']]

TypeError: list indices must be integers or slices, not str